<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# вставьте сюда параметры подключения из юнита 
# 1. Работа с базой данных из Python 


In [ ]:
conn = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

NameError: name 'psycopg2' is not defined

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1= f'''SELECT count(*) FROM public.vacancies'''



In [ ]:
#Результат здесь
with conn.cursor() as cur:
    try:
        cur.execute(query_3_1)
        result = cur.fetchone()
        print(f"Количество вакансий: {result[0]}")
    except:
        print("Таблица public.vacancies тоже не существует")

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
#запрашиваем количетсво работодателей и импортируем их  из  таблицы employers
query_3_2 = f'''select count(*) from public.employers'''


In [ ]:
# результат запроса
with conn.cursor() as cur: 
    try:
        cur.execute(query_3_2)
        result=cur.fetchone()
        print(f"Количество работадателей: {result[0]}")
    except:
        print("Таблица не существует")

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
#запрашиваем количетсво регионов и импортируем их  из  таблицы areas
query_3_3 = f'''select count(*) from public.areas''' 


In [ ]:
# результат запроса
with conn.cursor() as cur:
    try:
        cur.execute(query_3_3)
        result=cur.fetchone()
        print(f"Количество регионов: {result[0]}")
    except:
        print("Таблица не существует")

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
#запрашиваем информацию о кол0ве сфер деятельности из  таблицы industries
query_3_4= f'''
    select 
        count(*) 
    from public.industries
'''


In [ ]:
# результат запроса
with conn.cursor() as cur:
    try:
        cur.execute(query_3_4)
        result= cur.fetchone()
        print(f"Количество сфер деятельности: {result[0]}")
    except:
        print('Таблица не существует')

***

*Данные:*\
Количество вакансий 49194\
Количество работадателей 23501\
Количество региоов 1362\
Количество сфер деятельности 294

**Выводы по предварительному анализу:**
1. 49194 вакансии,  это большой обьем данных, что может говорить о активном рынке труда 
2. Работадатели в среднем выставляют 1-2 вакансии
3. Отношение количества сфер деятельносли к вакансиям говорит,  что в среднем на каждую сферу может приходится 167 вакансий
4. 1362 региона говорит о том, что работадатели имеют большую географическую представленность
5. На каждый регион приходится примерно 36 вакансий и 17 работадателей , это может говорить о том, что есть доминирующие регионы и регионы в которых могут быть до 1 размещенной вакансии или работадателя 



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса 
# запрашиваем информацию о кол-ве вакансий в каждом регионе 
query_4_1 = f'''
    select 
        a.name area, --регион
        count(*) cnt  --количество вакансий
    from public.areas as a 
            join public.vacancies as v on v.area_id=a.id
    group by 1 -- групируем по названию региона 
    order by 2 desc -- сортируем по кол-ву вакансий в порядке убывания 
         '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_1, conn)
df
    

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2= f''' 
        select 
                count(*) count_salary -- кол-во вакансий 
        from public.vacancies v 
        where (v.salary_from is not null) or (v.salary_to is not null) -- ищем только те у которых заполнено хоть одно поле с зп 
        '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_2, conn)
df

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f''' 
     select 
          avg(v.salary_from) mean_salary --  среднее по нижней границу зп вилки
     from public.vacancies v -- импортируем из  таблицы с вакансиями 
     
     union all  -- обьединяем  данные  с верней вилкой 
     
     select 
          avg(v.salary_to) mean_salary --  среднее по верхней границе зп вилки 
     from public.vacancies v -- импортируем из  таблицы с вакансиями 
     '''
        

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_3, conn)

round(df)

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''
    select 
        v.schedule schedule, -- тип графика
        v.employment employment, -- тип трудоустройства
        count(v.employment) count_employment -- количество для каждого сочетания 
    from public.vacancies v -- импортируем библиотеку с вакансиями
    group by 1,2 -- группируем по типу графика и трудоуствойства 
    order by 3 desc -- сотрируем по убыванию кол-ва 
    '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_4 , conn)

df

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f'''
    select 
        experience, --  требуемый опыт 
        count(*) -- количество вакансий
    from public.vacancies 
    group by 1 -- групируем по опыту 
    order by 2 -- сортируем по возрастанию  кол-а вакансий
    '''

In [ ]:
# результат запроса
df = pd.read_sql_query( query_4_5 , conn)

df

***

*Данные:*

1. Самое большое количество вакансий сосредоточено в городе МСК
2. Около 70 городов имеют вакансий более 100(22 из  них  более 500 вакансий на город)(9%)
3. Около 90% регионов имеют вакансий в среднем 1-2
4. Основная масса вакансий находится в первой десятке городов
5. Примерно у 50% вакансий не указана з\п
6. Средние значение з\п находятся в диапазоне от 71000 до 110000
7. Основная часть работодателей указывают график и тип трудоустройства полный день, полная занятость
8. К первой пятерке относятся полный день\полная занятость, удаленная работа\полная занятость, гибкий график\полная занятость, удаленная работа\частичная занятость
9. Чаще всего работодатели запрашивают полную занятость
10. Опыт работы более 6 лет имеют наименьшее значение
11. Самыми возтребованными сотрудниками являются люди с опытом от года до 3 лет
12. опыт работы 1-3 года 26152, 3-6 14511, без опыта 7197, более 6 лет 1337

# **Выводы по детальному анализу вакансий:**


*Общая характеристика рынка:*\
Централизация и региональная раздробленность
Выводы из пунктов 1-4 рисуют картину гиперцентрализованного рынка с ярко выраженным разрывом между столицей и регионами.\
Абсолютное доминирование Москвы (МСК):\
Гипотеза: На Москву приходится 10% и более всех вакансий платформы. Такой разрыв обусловлен тем, что в Москве сосредоточены головные офисы крупнейших компаний, IT-гигантов, банков, ритейла и стартапов, которые являются основными потребителями данных услуг.

*"Длинный хвост" городов и "вакуум" в регионах:*\
70 городов с >100 вакансий — это достаточно позитивный показатель, показывающий, что профессия востребована не только в миллионниках, но и в крупных региональных центрах (например, Новосибирск, Екатеринбург, Минск, Алматы, Красноярск и т.д.).\
90% регионов имеют 1-2 вакансии — означает, что за пределами топ-70 городов рынок труда практически отсутствует. Для специалиста в небольшом городе это означает либо удаленную работу, либо смену профессии/переезд.

*Концентрация в первой десятке:*\
Это прямое следствие пункта 1. Первая десятка — это Москва, Санкт-Петербург (SPB) и, с большим отставанием, города-миллионники. Вероятно, на топ-10 городов приходится 48% всего рынка вакансий.

**Сводный вывод по географии: Рынок работы для данных специалистов — это рынок больших городов. Карьерные возможности кардинально отличаются в зависимости от геолокации. Компаниям из регионов, вероятно, сложно конкурировать за кадры с столичными предложениями, особенно с возможностью удаленной работы.**

*Анализ зарплатных ожиданий:*\
50% вакансий без указания ЗП:\
Это огромная проблема для соискателя. Причины:
1. Переговоры: Работодатель хочет обсудить ЗП индивидуально, исходя из опыта кандидата.
2. Конкуренция: Не желает "пугать" высокими цифрами нерелевантных кандидатов или "отпугивать" низкими сильных специалистов.
3. Низкая ЗП: Часто за молчанием скрывается неконкурентный уровень оплаты.

Средняя вилка: 71к - 110к руб., может быть сильно завышена несколькими предложениями с очень высоким окладом.\
Вилка в 40k рублей говорит о высокой дифференциации зарплат в профессии. На итоговый оклад сильно влияет:\
Город: В Москве вилка может быть 110-150к, в региональном центре — 40-80к.

*Условия труда:*\
 Полная занятость и формат работы:\
Полный день/Полная занятость — это стандарт де-факто. Это говорит о том, что работа является основной и требует полного погружения.\
Вторая позиция — "удаленная работа/полная занятость" — это ключевой тренд. Рынок официально закрепил удаленку как массовый и легитимный формат. Это главная возможность для специалистов из регионов.\
Появление гибкого графика и частичной занятости в топе говорит о развитии рынка: есть запрос на более свободные формы сотрудничества, возможно, для совмещения projects или работы с фрилансерами.

*Требования к опыту: Рынок для мидлов*\
Опыт работы — самый важный вывод для соискателя:\
Явный запрос на "Мидлов" (1-6 лет опыта): 26 152 (1-3 года) + 14 511 (3-6 лет) = 40 663 вакансии. Это более 70% от всех вакансий, где указан опыт. Компании ищут специалистов, которые уже прошли начальный этап, но еще не доросли до дорогих сеньор-позиций. Они обладают актуальными знаниями и могут самостоятельно решать задачи.\
Спрос на джуниоров (без опыта): 7 197 вакансий — это значительное число. Оно говорит о том, что компании готовы растить кадры внутри, инвестировать в обучение. Это хороший вход в профессию для новичков.\
Низкий спрос на сеньоров (6+ лет): 1 337 вакансий — это не значит, что они не нужны. Это значит, что:

1. Таких вакансий объективно меньше, так как один сеньор может руководить работой нескольких мидлов.
2. Часто "сеньор" — это уже не просто специалист, а руководитель команды.

## ***Общий итоговый портрет рынка:***
1. Для кого рынок: Для специалистов среднего уровня (мидлов) с опытом от 1 до 6 лет, готовых к полной занятости.
2. Где работать: Лучшие возможности — в Москве и крупных городах-миллионниках. Для жизни в малом городе — единственный вариант это удаленка.
3. На что рассчитывать в зарплате: При поиске нужно быть готовым к торгу, так как в половине случаев вилки нет. Ориентировочный диапазон для мидла — вокруг 100к, но с огромным разбросом.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1= f'''
    select 
        e.name  name_employers, -- названия компаний
        count(*) count_vacancies --  кол-во вакансий 
    from public.vacancies v
    join public.employers e on v.employer_id=e.id -- присоединяем таблицу с работодателями к вакансиям по id  работодателя 
    group by 1 --  групируем по названию компании
    order by 2 desc -- сортируем по кол-ву вакансий в порядке убывания
    limit 5 -- выводи топ 5 компаний лидирующих  по кол-ву вакансий 
    '''

In [ ]:
# результат запроса
df = pd.read_sql_query( query_5_1 , conn)
df

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''
    select 
        a.name, -- регионы 
        count(v.id) cnt_vacancies,  -- кол-во вакансий 
        count( e.id) cnt_employers -- кол-во работодателей 
    from public.areas a
    left join public.vacancies v on a.id = v.area_id -- присоединяем использую left join  таблицу с вакансиями по столбцу id региона 
    left join public.employers e on a.id = e.area -- присоединяем таблицу с работодателями по столбцу id региона
    group by a.name -- групируем по названию региона 
    having count(v.id) = 0 = доп условие,  выводим только тех  у кого нет вакансий
    order by 3 desc -- сортируем по кол-ву работодателей в порядке убывания
    limit 1 --выводим регион с самым большим ко-вом работодателей без  вакансии
    
    
    -- если необходимо посчитать топ 3 региона у которых не указан
    --работодатель то 
    --having count(e.id)= 0 
    --order by 2 desc 
    --limit 3
'''


In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_2, conn)

df

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3= f'''
    select
        e.name employer, --  работодатели
        count(distinct v.area_id) count_region -- кол-во регионов с вакансиями 
    from public.employers e
    left join public.vacancies v on e.id= v.employer_id -- присоединяем таблицу с вакансиями используя left join
    group by 1 --групируем по работодателю
    order by 2 desc сортируем по кол-ву регионов в порядке убывания 
'''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_3, conn)

df

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f''' 
    SELECT 
    COUNT(*) -- кол-во работодателей
FROM public.employers e -- импортируем таблицу с работодателями
LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id --присоединяем таблиру со сфера деятельности используя left join 
WHERE ei.employer_id IS NULL -- при условии сфера деятельности =  0
'''

In [ ]:
# результат запроса
df= pd.read_sql_query(query_5_4 , conn)
df

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = f'''
    select 
        e.name, --  название компании 
        count(industry_id) -- считаем количество сфер деятельности 
    from public.employers e -- импортируем таблицу с работодателями 
    join public.employers_industries ei on e.id=ei.employer_id -- присоединяем используя join таблицу со сферами деятельности 
    group by 1 -- группируем по названию  компании
    having count(industry_id) = 4 -- учитываем только те компании у которых  ровно 4 сферы деятельности
    order by 1  -- сортируем по названию в алфавитном порядке 
    offset 2 limit 1 -- выводим компанию которая находится на 3 месте 
    
'''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_5,conn)
df

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f''' 
    SELECT 
    COUNT(*) -- количество работодателей
    FROM public.employers_industries ei  --импортируем библиотеку  со сферами деятельности 
    join public.industries i on ei.industry_id=i.id -- присоединяем библиотеку  с вакансиями 
    WHERE i.name='Разработка программного обеспечения' -- учитываем только те где указана сфера разработка программного обеспечения
    '''

In [ ]:
# результат запроса
df=pd.read_sql_query(query_5_6, conn)
df

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
#импортируем библиотеки для анализа 
import requests
from bs4 import BeautifulSoup 


# Получаем список городов-миллионеров из Википедии
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' 

soup = BeautifulSoup(requests.get(url).text, 'html.parser')
table = soup.find('table', class_='standard')

million_cities = []
if table:
    # Находим индекс города
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
    city_idx = headers.index('Город')
    
    # Собираем города
    million_cities = [row.find_all(['td', 'th'])[city_idx].text.strip() 
                     for row in table.find_all('tr')[1:] 
                     if len(row.find_all(['td', 'th'])) > city_idx]



***

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Получаем список городов-миллионеров из Википедии
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')
table = soup.find('table', class_='standard')

million_cities = []
if table:
    # Находим индекс города
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
    city_idx = headers.index('Город')
    
    # Собираем города
    million_cities = [row.find_all(['td', 'th'])[city_idx].text.strip() 
                     for row in table.find_all('tr')[1:] 
                     if len(row.find_all(['td', 'th'])) > city_idx]

# Проверяем, что список городов не пустой
if million_cities:
    # Формируем SQL запрос с фильтрацией по городам-миллионерам
    cities_sql = ', '.join([f"'{city}'" for city in million_cities])
    
    query_5_7 = f'''
        -- Запрос для регионов-миллионников
        SELECT 
            a.name as region, --регион
            COUNT(v.id) as vacancy_count  -- кол-во вакансий 
        FROM public.vacancies v 
        JOIN public.areas a ON v.area_id = a.id -- присоединяем регионы к таблице с вакансиями 
        JOIN public.employers e ON v.employer_id = e.id -- присоединяем работодателей к таблице с вакансиями 
        WHERE e.name = 'Яндекс' -- выбираем только компанию яндеск 
        AND a.name IN ({cities_sql}) --  и регионы милионники из  списка 
        GROUP BY a.name -- групируем по региону 
        
        UNION ALL -- обьединяем две таблицы присоединив общее кол-во вакансий в городах  милионниках
        
        -- Общее количество вакансий в городах-миллионниках
        SELECT 
            'Total' as region, -- название общего колва вакансий
            COUNT(v.id) as vacancy_count -- кол-во вакансий
      JOIN public.areas a ON v.area_id = a.id -- присоединяем регионы к таблице с вакансиями 
        JOIN public.employers e ON v.employer_id = e.id -- присоединяем работодателей к таблице с вакансиями 
        WHERE e.name = 'Яндекс' -- выбираем только компанию яндеск 
        AND a.name IN ({cities_sql})  --  и регионы милионники из  списка 
    '''


In [ ]:
# выводы по анализу работодателей
df= pd.read_sql_query(query_5_7 , conn)
df

## Выводы по Анализу работодателей:

*1. Топ-5 компаний по количеству вакансий*
1.	Яндекс	1933
2.	Ростелеком	491
3.	Тинькофф	444
4.	Сбер	428
5.	Газпром нефть	331

***Наблюдается значительное превосходсво IT-компаний и госкорпораций. Разрым между Яндексом и другими компаниями,  говорит о доминирующем положении Яндекса на рынке труда***

*2. Регионы без вакансий, но с наибольшим количеством работодателей (топ-3)*
1.	Россия	410
2.	Казахстан	207
3.	Московская обл	75

***Это может говорить о возможных ошибках в заполнении данных***

*3. Топ-3 региона, где не указан работодатель (всего таких регионов: 708)*
1.	Кишинев	23
2.	Лобня	16
3.	Мирный (Якутия)	12

***Это может говорить,  что некоторые вакансии имеют в себе анонимные вакансии либо так же ошибку в данных***

*4. Количество регионов на каждого работодателя (топ-3)*
1.	Яндекс	181
2.	Ростелеком	152
3.	Спецремонт	116\
Остальные работодатели представлены менее чем в 100 регионах.

***Основным работодателем по количеству регионов является Яндекс, что говорит о ее представленности на национальном уровне, сюда так же можно включить "ростелеком" и "спецремонт"***

*5. Работодатели без указания сферы деятельности*\
Количество таких работодателей: 8419

***Около 36% всех  работодателей не указывают сферы деятельности,  что может говорить о проблемах  с заполнением информации о вакансии***

***Специализация компаний:***
- 3553 работодателя указали сферу "Разработка программного обеспечения"
- 2415 компаний работают в 4+ сферах деятельности
- 1136 компаний указали ровно 4 сферы деятельности


*9. Города-миллионники с вакансиями и их количество*
1.	Москва	54
2.	Санкт-Петербург	42
3.	Екатеринбург	39
4.	Нижний Новгород	36
5.	Новосибирск	35
6.	Воронеж	32
7.	Краснодар	30
8.	Владивосток	27
9.	Самара	26
10.	Уфа	26

***Наблюдается четкая корреляция между размером города и количеством вакансий,  что подтверждает централизацию рынка труда в крупных  компаниях***

# Ключевые выводы:

1. Доминирование IT-сектора: Компании Яндекс и Тиньков лидируют по количеству вакансий и георгафичскому охвату 
2.  Проблемы данных: Значительная часть информации о работодатели заполнена не полностью(сферы деятельности и работодатель)
3.  Географическая концентрация: Большинство вакансий расположено в крупных  городах,  при этом федеральные компании имеют широкое региональное присутствие 
4. Большое количество работодателей работают в 4 и более сферах  деятельности ,  что говорит о:
- диверсификаци бизнеса
- широком профиле услуг
- холдинговой структуре 



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1= f''' 
    SELECT 
    --  считаем кол-во вакансий имеющих  отношение к даннным 
        count(*) 
    FROM public.vacancies v
    WHERE lower(v.name) LIKE '%data%' or 
        lower(v.name) LIKE '%данн%' 

'''

In [ ]:
# результат запроса
df=pd.read_sql_query(query_6_1, conn)
df

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
query_6_2 = f'''
-- считаем кол-во вакансий имеющих  отношение в DS 
SELECT COUNT(*)
FROM public.vacancies v
JOIN public.areas a  ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
WHERE 
--по критерию специалист уровня junior
    (v.name ILIKE '%junior%' OR
     v.employment ILIKE '%стажировка%' OR
     v.experience ILIKE '%нет опыта%')
  AND 
    (
      (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%html%')
      OR v.name ILIKE '%data scientist%'
      OR v.name ILIKE '%data science%'
      OR v.name ILIKE '%исследователь данных%'
      OR v.name ILIKE '%machine learning%'
      OR v.name ILIKE '%машинн%обучен%'
    )
    '''

In [ ]:
# результат запроса

df=pd.read_sql_query(query_6_2, conn)
df

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''
-- считаем кол-во вакансий для DS 
SELECT COUNT(*)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
WHERE 
-- только тех, чей  ключевой навык sql or postgres
  (v.key_skills ILIKE '%SQL%' or
   v.key_skills ILIKE '%postgres%')
    
  AND 
    (
      (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%html%')
      OR v.name ILIKE '%data scientist%'
      OR v.name ILIKE '%data science%'
      OR v.name ILIKE '%исследователь данных%'
      OR v.name ILIKE '%machine learning%'
      OR v.name ILIKE '%машинн%обучен%'
    )
    '''


In [ ]:
# результат запроса
df=pd.read_sql_query(query_6_3, conn)
df

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''
-- считаем кол-во вакансий для DS у которых  ключевой навык python
SELECT COUNT(*)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
WHERE 
    v.key_skills ILIKE '%python%'
    
    AND 
    (
      (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%html%')
      OR v.name ILIKE '%data scientist%'
      OR v.name ILIKE '%data science%'
      OR v.name ILIKE '%исследователь данных%'
      OR v.name ILIKE '%machine learning%'
      OR v.name ILIKE '%машинн%обучен%'
    )
    '''

In [ ]:
# результат запроса
df=pd.read_sql_query(query_6_4, conn)
df

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:

# текст запроса
query_6_5 = f'''
SELECT 
-- среднее кол-во навыков указывающих для вакансий DS
ROUND(AVG(CHAR_LENGTH(v.key_skills) - CHAR_LENGTH(replace(v.key_skills, CHR(9), '')) + 1), 2) AS "Количество Слов"

FROM vacancies v
WHERE
        (v.name ILIKE '%data scientist%' OR
    v.name ILIKE '%data science%' OR
    v.name ILIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' AND v.name NOT ILIKE'%html%') OR
    v.name ILIKE '%machine learning%' OR
    v.name ILIKE '%машинн%обучен%') AND
    (v.key_skills IS NOT NULL AND v.key_skills != '')
'''

In [ ]:
# результат запроса+
df=  pd.read_sql_query(query_6_5 , conn)
df

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:

# текст запроса
query_6_6 = f'''
(SELECT 
-- высчитываем среднюю зп для каждого типа опыта работы 
    experience,
    ROUND(AVG(coalesce(salary_from , salary_to , 0) + coalesce(salary_to , salary_from , 0)) /2) avg_salary

FROM vacancies v
WHERE 
    (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
    --and
    --v.experience= 'От 3 до 6 лет' -- доп условие,  выводим только данные для DS с опытом 3-6 лет
    AND
    (v.name ILIKE '%data scientist%' OR
    v.name ILIKE '%data science%' OR
    v.name ILIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' AND v.name NOT ILIKE'%html%') OR
    v.name ILIKE '%machine learning%' OR
    v.name ILIKE '%машинн%обучен%') 

GROUP BY experience)
'''


In [ ]:
# результат запроса
df=  pd.read_sql_query(query_6_6 , conn)
df

***



### Предметный анализ вакансий в сфере Data Science (DS)

#### 1. **Объем рынка и его структура**
*   **Вакансии, имеющие отношение к данным:** 1771
    *   Это ядро рынка, включающее Data Scientists, Data Analysts, Data Engineers и т.д.
*   **Вакансии для начинающих дата-сайентистов:** 51
    *   **Вывод:** Это крайне мало (~2.9% от всех "data-вакансий"). Рынок ориентирован на опытных специалистов. Для джуниоров входной барьер очень высокий.

#### 2. **Ключевые навыки**
*   **SQL/PostgreSQL:** 229 вакансий
    *   Это **must-have** навык. Владение базами данных и языком запросов SQL требуется практически каждому специалисту, работающему с данными.
*   **Python:** 357 вакансий
    *   **Вывод:** Python — лидер и стандарт в индустрии. Его важность даже выше, чем у SQL (охват ~20% от всех data-вакансий против ~13% для SQL).

#### 3. **Требования к квалификации**
*   **Среднее количество ключевых навыков на вакансию:** 6.41
    *   **Вывод:** Это очень высокий показатель. Работодатели ищут **универсальных специалистов**. От кандидата ожидается владение целым комплексом инструментов: язык программирования (Python/R), базы данных (SQL), библиотеки для анализа (Pandas, NumPy), машинное обучение (Scikit-learn, TensorFlow), визуализация (Tableau, PowerBI), soft skills.

#### 4. **Заработные платы в зависимости от опыта**
Зарплатная вилка четко коррелирует с опытом, показывая **кратный рост** доходов:
| Опыт работы         | Средняя заработная плата | Рост относительно предыдущего уровня |
|---------------------|---------------------------|---------------------------------------|
| **Нет опыта (Junior)** | 75 000 руб.               | -                                     |
| **1-3 года (Middle)**  | 140 000 руб.              | +87%                                  |
| **3-6 лет (Senior)**   | 243 000 руб.              | +74%                                  |

**Вывод:**
1.  **Высокая ценность опыта:** Инвестиция в профессиональное развитие в DS окупается очень быстро. Разрыв между Junior и Senior более чем в 3 раза.
2.  **"Зарплатный прыжок"** происходит на этапе перехода с Junior на Middle-уровень (почти +90% к доходу). Это мотивирует начинающих специалистов активно набираться опыта.

---

### Портрет вакансии и кандидата в Data Science

**Типичная вакансия в DS требует:**
*   **Опыта работы** от 1 года до 3 лет (Middle-уровень).
*   **Владение широкими навыками** (в среднем 6+ навыков), ключевые из которых — **Python** и **SQL**.
*   **Готовности платить** среднюю зарплату в **140-240 тыс. руб.** для специалистов с опытом.

**Портрет идеального кандидата:**
*   **Middle/Senior-level** с опытом от 3 лет.
*   **Универсал**, а не узкий специалист.
*   С сильной технической базой (кодирование на Python, работа с БД).
*   С портфолио реализованных проектов.

---

###  Общий вывод:

*   **Для работодателей:** Рынок испытывает дефицит готовых Middle/Senior-специалистов, что объясняет высокий уровень зарплат. Требования к кандидатам завышены (6+ навыков), что может удлинять срок поиска.
*   **Для начинающих специалистов (Junior):** Вход в профессию сложен из-за малого количества джуниор-вакансий. Необходимо:
    1.  **Фокусироваться на основе:** делать упор на **Python** и **SQL**.
    2.  Собирать **портфолио** с реальными проектами.
    3.  Рассматривать смежные должности (например, Data Analyst) для набора опыта.
*   **Для опытных специалистов:** Рынок крайне благоприятен. Карьерный рост и рост заработной платы — одни из самых быстрых в IT-секторе.

**Итог:** Рынок Data Science — это зрелый, высокооплачиваемый сегмент с высокими требованиями к квалификации и огромным потенциалом для роста для тех, кто готов инвестировать в свое образование и опыт.

# Общий вывод по проекту

### 🎯 Ключевые выводы: Общая картина рынка вакансий

#### 1. **Высокая концентрация и централизация**
Рынок труда **крайне неоднороден** и сконцентрирован в нескольких точках:
*   **Географически:** В крупных городах-миллионниках (Москва, СПб), где сосредоточено большинство вакансий. >90% регионов имеют лишь по 1-2 вакансии.
*   **Среди работодателей:** Небольшая группа крупных компаний (Яндекс, Сбер, Тинькофф, Ростелеком) формирует значительную долю всего рынка вакансий.
*   **В сферах деятельности:** Доминирует **IT-сектор**, особенно направление **Data Science** и разработки.

#### 2. **Доминирование IT и Data Science**
*   **Data Science** — один из самых перспективных и высокооплачиваемых сегментов.
    *   **Высокие требования:** От кандидатов ожидают владения 6+ навыками одновременно (Python, SQL, ML и др.).
    *   **Высокие зарплаты:** Зарплата кратно растет с опытом: с ~75к руб. для juniors до ~240к руб. для seniors.
    *   **Сложный вход:** Для начинающих специалистов (junior) вакансий очень мало (~51), рынок ориентирован на опытных профессионалов.

#### 3. **Проблема качества данных**
Значительная часть данных на платформах — **неполная или неточная**, что искажает аналитику:
*   **У 50% вакансий не указана зарплата.**
*   **36% работодателей не указали сферу деятельности.**
*   **В 708 регионах** работодатель в вакансиях не указан (анонимные вакансии или ошибки).
*   Это создает серьезные барьеры для соискателей в поиске и оценке предложений.

#### 4. **Структура работодателей: гиганты и "длинный хвост"**
*   **Ядро (~7% рынка):** Крупные федеральные компании с широкой филиальной сетью (более 100 регионов) и большим количесвом вакансий.
*   **"Длинный хвост" (~93% рынка):**  Мелких работодателей (МСП, ИП) с 1-2 вакансиями, часто с плохо заполненными профилями.

#### 5. **Тренды занятости**
*   **Стандарт:** Формат **полный день/полная занятость** остается основным.
*   **Тренд:** **Удаленная работа** и **гибкий график** прочно вошли в топ-5 форматов, что отражает изменение рынка после пандемии.

### 💡 Итоговый портрет рынка

**Общая рекомендация:**
*   **Соискателям:** Сфокусироваться на развитии hard skills (Python, SQL) и сборе портфолио. Рассматривать удаленные вакансии из столичных компаний.
*   **Работодателям:** Повышать прозрачность вакансий (указывать зарплату), активнее работать с junior-талантами и развивать удаленные форматы для доступа к региональным кадрам.
*   **Аналитикам:** Учитывать неполноту данных при построении любых выводов.